## 条件过滤

1. 读取本地数据
2. 基本面因子（过滤指数成分停牌涨跌停）
3. 技术因子与事件驱动（可买可卖）

## 1_读取本地数据

In [1]:
import jaqs_fxdayu
jaqs_fxdayu.patch_all()
from jaqs.data import DataView
from jaqs.data import RemoteDataService
import os
import numpy as np
import warnings

warnings.filterwarnings("ignore")

D:\Anaconda3\lib\importlib\__init__.py:126: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  return _bootstrap._gcd_import(name[level:], package, level)
D:\Anaconda3\lib\site-packages\matplotlib\__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
dataview_folder = './JAQS_Data/hs300'
dv = DataView()
dv.load_dataview(dataview_folder)

Dataview loaded successfully.


## 3_过滤停牌涨跌停（可买可卖）

mask_index_member为要过滤的为True

can_enter与can_exit皆为可交易为True

In [3]:
from jaqs_fxdayu.util import dp
from jaqs.data.dataapi import DataApi

def mask_index():
    df_index_member = dv.get_ts('index_member')
    mask_index_member = df_index_member==0 #定义信号过滤条件-非指数成分
    return mask_index_member

def limit_up_down():
    # 定义可买卖条件——未停牌、未涨跌停
    trade_status = dv.get_ts('trade_status')
    mask_sus = trade_status == u'停牌'
    # 涨停
#     dv.remove_field('up_limit')
    dv.add_formula('up_limit', '(close - Delay(close, 1)) / Delay(close, 1) > 0.095', is_quarterly=False, add_data=True)
    # 跌停
#     dv.remove_field('down_limit')
    dv.add_formula('down_limit', '(close - Delay(close, 1)) / Delay(close, 1) < -0.095', is_quarterly=False, add_data=True)
    can_enter = np.logical_and(dv.get_ts('up_limit') < 1, ~mask_sus) # 未涨停未停牌
    can_exit = np.logical_and(dv.get_ts('down_limit') < 1, ~mask_sus) # 未跌停未停牌
    return can_enter,can_exit

In [4]:
mask_index_member = mask_index()
can_enter,can_exit = limit_up_down()

Field [up_limit] is overwritten.
Field [down_limit] is overwritten.


In [5]:
# dv.remove_field('mask_index_member')
# dv.remove_field('can_enter')
# dv.remove_field('can_exit')

In [6]:
dv.append_df(mask_index_member, 'mask_index_member')
dv.append_df(can_enter, 'can_enter')
dv.append_df(can_exit, 'can_exit')

Field [mask_index_member] is overwritten.
Field [can_enter] is overwritten.
Field [can_exit] is overwritten.


In [7]:
dv.save_dataview('./JAQS_Data/hs300')


Store data...
Dataview has been successfully saved to:
E:\QTC\PythonQTC\course\3_JAQS\JAQS_Data\hs300

You can load it with load_dataview('E:\QTC\PythonQTC\course\3_JAQS\JAQS_Data\hs300')


In [8]:
print(dv.get_ts('mask_index_member').head())

symbol      000001.SZ  000002.SZ  000008.SZ  000009.SZ  000012.SZ  000021.SZ  \
trade_date                                                                     
20120105        False      False       True      False      False      False   
20120106        False      False       True      False      False      False   
20120109        False      False       True      False      False      False   
20120110        False      False       True      False      False      False   
20120111        False      False       True      False      False      False   

symbol      000024.SZ  000027.SZ  000031.SZ  000039.SZ    ...      601998.SH  \
trade_date                                                ...                  
20120105        False      False       True      False    ...          False   
20120106        False      False       True      False    ...          False   
20120109        False      False       True      False    ...          False   
20120110        False      False       